# A Real-world Example Using HBV-SASK Hydrological Model

## Installation (from source)

The latest version of the varstool sensitivity and uncertainty package can always be retrieved from its GitHub repo using the following commands:

note: if the loading bars are not working, simply turn them off or install a lower version of pandas by doing the following:
`!pip install pandas==1.2.5`

In [ ]:
# !rm -rf vars-tool
# !git clone https://github.com/vars-tool/vars-tool
# !pip install vars-tool/.
# !pip install seaborn

## HBV-SASK Model - A Real-world Sensitivity Analysis Problem

The HBV-SASK Model is a real-world hydrological model that has been developed within the [**"Watershed Systems Analysis and Modelling Lab"**](https://www.samanrazavi.com) for educational purposes. In this example, we analyze the sensitivity of each parameter of the HBV-SASK model to the simulated streamflow (i.e., output) using the state-of-the-art varstool Python package.

HBV-SASK utilizes daily time-series of precipitation and temperature and produced many outputs, such as streamflow, evapotransipration, state variables, etc.

For more information see: Razavi, S., Sheikholeslami, R., Gupta, H. V., & Haghnegahdar, A. (2019). ***VARS-TOOL: A toolbox for comprehensive, efficient, and robust sensitivity and uncertainty analysis.*** Environmental modelling & software, 112, 95-107. doi: https://doi.org/10.1016/j.envsoft.2018.10.005

In [1]:
from varstool import TSVARS, Model

# Loading the "HBV_SASK" model that is included in this package
from varstool.example_models import HBV_SASK

exp_1_model = Model(HBV_SASK)

Let's just see how the model works. The object "Model" is an standard way of wrapping different functions that are going to be used by varstool. Therefore, users must communicate with their function of interest using this simple wrapper class. The arguments of this class can be quickly viewed by the following command:

In [2]:
Model?

Init signature: Model(func: Callable = None, unknown_options: Dict[str, Any] = {}) -> None
Docstring:     
Description:
------------
A wrapper class to contain various models and functions
to be fed into VARS and its variations. The models can be
called by simply calling the wrapper class itself.


Parameters:
-----------
:param func: function of interest
:type func: Callable
:param unknown_options: a dictionary of options with keys as
                        parameters and values of parameter
                        values.
:type unknown_options: dict
File:           c:\users\corde\anaconda3\lib\site-packages\varstool\varstool.py
Type:           type
Subclasses:     


To quickly test the HBV-SASK model with some random numbers as its input, follow the cell below:

In [3]:
import pandas as pd

params={#name  #value
       'TT'   : -4.00,
       'C0'   : 0.00,
       'ETF'  : 0.00,
       'LP'   : 0.10,
       'FC'   : 50.0,
       'beta' : 1.00,
       'FRAC' : 0.10,
       'K1'   : 0.05,
       'alpha': 1.00,
       'K2'   : 0.00,
       'UBAS' : 1.00,
       'PM'   : 0.50,
}

HBV-SASK returns many outputs; for the sake of simplicity, here we just use the simulated streamflow as the chosen model response (shown below). HBV-SASK returns a time-series of streamflow but, we are using GVARS which only accepts non time-series inputs right now. The time-series version of this algorithm is detailed in the following publication:

* Gupta, H. V., & Razavi, S. (2018). Revisiting the basis of sensitivity analysis for dynamical earth system models. Water Resources Research, 54(11), 8692-8717. doi: https://doi.org/10.1029/2018WR022668

In [4]:
# custome output selection - we are just now using the last 20 time-steps
def custom_HBV_SASK(x):
    # on MS Windows machines, import the original library again in
    # the body of the function to be wrapped, if any.
    # No need to import again on *nix machines
    from varstool.example_models import HBV_SASK
    
    return HBV_SASK(x)[0]['Q_cms'][-20:]

exp_1_model = Model(custom_HBV_SASK)

# and testing the output
exp_1_model(pd.Series(params))

2011-12-12    0.337263
2011-12-13    0.320400
2011-12-14    0.304380
2011-12-15    0.289161
2011-12-16    0.274703
2011-12-17    0.260968
2011-12-18    0.669035
2011-12-19    0.635583
2011-12-20    0.603804
2011-12-21    0.573614
2011-12-22    0.544933
2011-12-23    0.517687
2011-12-24    0.491802
2011-12-25    0.467212
2011-12-26    0.443851
2011-12-27    0.421659
2011-12-28    0.400576
2011-12-29    0.738123
2011-12-30    0.892118
2011-12-31    0.847512
Name: Q_cms, dtype: float64

In [5]:
# setting up a TSVARS experiment for the HBV-SASK model

exp_1 = TSVARS(num_stars=2,
               parameters={  #lb     #ub
                   'TT'   : [-4.00, 4.00],
                   'C0'   : [0.00 , 10.0],
                   'ETF'  : [0.00 , 1.00],
                   'LP'   : [0.00 , 1.00],
                   'FC'   : [50.0 ,500.0],
                   'beta' : [1.00 , 3.00], 
                   'FRAC' : [0.10 , 0.90],
                   'K1'   : [0.05 , 1.00],
                   'alpha': [1.00 , 3.00],
                   'K2'   : [0.00 , 0.05],
                   'UBAS' : [1.00 , 3.00],
                   'PM'   : [0.50 , 2.00],
               },
               delta_h=0.1,
               ivars_scales=(0.1, 0.3, 0.5),
               model=exp_1_model,
               seed=1001,
               sampler='lhs',
               bootstrap_flag=False,
               grouping_flag=False,
               report_verbose=True,
               func_eval_method='parallel',
               vars_eval_method='parallel',
               vars_chunk_size=None,
          )

At any time, the status of the VARS analysis could be viewed by typing the name of the instance that is used to initiate the experiment. Here, it is `exp_1`:

In [6]:
exp_1

Star Centres: Loaded
Star Points: Not Loaded
Parameters: 12 paremeters set
Delta h: 0.1
Model: custom_HBV_SASK
Seed Number: 1001
Bootstrap: Off
Bootstrap Size: N/A
Bootstrap CI: N/A
Grouping: Off
Number of Groups: None
Function Evaluation Method: parallel
TSVARS Evaluation Method: parallel
TSVARS Chunk Size: N/A
Verbose: On
TSVARS Analysis: Not Done

From the status report, it could be viewed that the `star_centres` are fully sampled (via `lhs`) and loaded. Meanwhile, $\Delta$h (`delta_h`) is set to 0.1, `seed` is set to 1001, and 12 parameters are fully set to begin the VARS sensitivity analysis.

Time-series VARS (TSVARS) comes with two computational methods to carry out sensitivity analysis, that are:
1. Serial and
2. Parallel.

It is worth noting that these computational methods are applicable to: **a**) while varstool runs the `Model` and **b**) when varstool computes common `VARS` indices. For the former `func_eval_method` method must be changed and for the latter `vars_eval_method` to either `'serial'` or `'parallel'`.

Furthermore, to save memory usage, the computations can be chunked into several segments to avoid memory leak. In this example, first a simple serial version without chunking is demonstrated. It should be noted that the parallel version are unstable and might result in unexpected errors.

Before carrying out a sensitivity analysis, let's check the loaded `star_centres`:

In [7]:
exp_1.star_centres

array([[0.74408722, 0.96041308, 0.09803003, 0.21526074, 0.01155678,
        0.09789096, 0.9911775 , 0.51015798, 0.85274302, 0.29022856,
        0.6763727 , 0.02056527],
       [0.15311609, 0.13253178, 0.55455094, 0.70552831, 0.73506341,
        0.56364827, 0.17640264, 0.11162101, 0.30676093, 0.98275457,
        0.42678384, 0.80294738]])

It could be seen that 2 star centres for 12 parameters of the HBV-SASK model are successfully produced. Now let's move on to the sensitivity analysis bit:

In [8]:
exp_1.run_online()

C:\Users\Corde\anaconda3\lib\site-packages\varstool\varstool.py:1881: UserWarning: Evaluating function in parallel mode is not stable yet. varstool currently uses `mapply` to parallelize function evaluations, see https://github.com/ddelange/mapply
  warnings.warn(


  0%|          | 0/240 [00:00<?, ?it/s]

building pairs:   0%|          | 0/20 [00:00<?, ?it/s]

VARS analysis:   0%|          | 0/10 [00:00<?, ?it/s]

You can see the list of chosen outputs in the `output` attribute of the VARS analysis experiment:

In [54]:
df = exp_1.gamma.unstack(level=0)
df2 = df.div(df.sum(axis=1), axis=0)
display(exp_1.gamma.aggregate)
# let him know the normalized average is always going to be the same for all h values
display(df2.sum(axis=1)/df2.shape[1])
display(df2)

param  h  
C0     0.1    1.638033
       0.2    1.833067
       0.3    2.034193
       0.4    2.294297
       0.5    2.632140
                ...   
beta   0.5    1.604701
       0.6    2.299489
       0.7    3.143297
       0.8    4.162663
       0.9    5.397008
Length: 108, dtype: float64

param  h  
C0     0.1    0.05
       0.2    0.05
       0.3    0.05
       0.4    0.05
       0.5    0.05
              ... 
beta   0.5    0.05
       0.6    0.05
       0.7    0.05
       0.8    0.05
       0.9    0.05
Length: 108, dtype: float64

ts         2011-12-12  2011-12-13  2011-12-14  2011-12-15  2011-12-16  \
param h                                                                 
C0    0.1    0.019971    0.017964    0.016172    0.014570    0.013133   
      0.2    0.019927    0.017924    0.016137    0.014538    0.013105   
      0.3    0.019626    0.017655    0.015895    0.014321    0.012910   
      0.4    0.019244    0.017311    0.015586    0.014043    0.012659   
      0.5    0.019010    0.017101    0.015397    0.013872    0.012506   
...               ...         ...         ...         ...         ...   
beta  0.5    0.018187    0.016326    0.014674    0.013201    0.011885   
      0.6    0.018183    0.016323    0.014670    0.013197    0.011882   
      0.7    0.018170    0.016310    0.014659    0.013187    0.011873   
      0.8    0.018145    0.016289    0.014640    0.013170    0.011857   
      0.9    0.018108    0.016256    0.014611    0.013144    0.011834   

ts         2011-12-17  2011-12-18  2011-12-19  2011-12-20  2011-12-21  \
param h                                                                 
C0    0.1    0.011844    0.206902    0.231874    0.110201    0.071365   
      0.2    0.011819    0.207159    0.232266    0.110246    0.071326   
      0.3    0.011643    0.207982    0.233447    0.110476    0.071318   
      0.4    0.011417    0.208799    0.234766    0.110831    0.071382   
      0.5    0.011279    0.209241    0.235621    0.111104    0.071445   
...               ...         ...         ...         ...         ...   
beta  0.5    0.010707    0.194960    0.227395    0.117166    0.076782   
      0.6    0.010704    0.194808    0.227297    0.117219    0.076835   
      0.7    0.010696    0.194863    0.227345    0.117213    0.076828   
      0.8    0.010682    0.195135    0.227548    0.117145    0.076757   
      0.9    0.010662    0.195653    0.227924    0.117007    0.076615   

ts         2011-12-22  2011-12-23  2011-12-24  2011-12-25  2011-12-26  \
param h                                                                 
C0    0.1    0.053341    0.042938    0.036020    0.030964    0.027024   
      0.2    0.053282    0.042877    0.035963    0.030911    0.026976   
      0.3    0.053204    0.042777    0.035861    0.030813    0.026885   
      0.4    0.053165    0.042700    0.035769    0.030719    0.026793   
      0.5    0.053148    0.042650    0.035706    0.030652    0.026727   
...               ...         ...         ...         ...         ...   
beta  0.5    0.057065    0.045508    0.037834    0.032274    0.027991   
      0.6    0.057106    0.045538    0.037857    0.032292    0.028005   
      0.7    0.057100    0.045534    0.037854    0.032290    0.028003   
      0.8    0.057046    0.045494    0.037823    0.032266    0.027984   
      0.9    0.056938    0.045413    0.037762    0.032218    0.027946   

ts         2011-12-27  2011-12-28  2011-12-29  2011-12-30  2011-12-31  
param h                                                                
C0    0.1    0.023818    0.021131    0.018833    0.016840    0.015093  
      0.2    0.023775    0.021093    0.018799    0.016809    0.015066  
      0.3    0.023691    0.021016    0.018729    0.016746    0.015008  
      0.4    0.023604    0.020935    0.018654    0.016677    0.014945  
      0.5    0.023541    0.020876    0.018599    0.016626    0.014899  
...               ...         ...         ...         ...         ...  
beta  0.5    0.024546    0.021688    0.019266    0.017180    0.015364  
      0.6    0.024557    0.021697    0.019273    0.017186    0.015369  
      0.7    0.024555    0.021696    0.019272    0.017185    0.015368  
      0.8    0.024540    0.021683    0.019261    0.017176    0.015360  
      0.9    0.024509    0.021657    0.019240    0.017158    0.015345  

[108 rows x 20 columns]

To view the produced variogram:

In [ ]:
exp_1.output['Gamma'].unstack(level=1) # unstack to make the dataframe more legible

## Aggregated Values

The aggregated values of the produced outputs are also possible by getting `.aggregate` of each value, for example:

In [11]:
exp_1.gamma.aggregate

param  h  
C0     0.1    1.638033
       0.2    1.833067
       0.3    2.034193
       0.4    2.294297
       0.5    2.632140
                ...   
beta   0.5    1.604701
       0.6    2.299489
       0.7    3.143297
       0.8    4.162663
       0.9    5.397008
Length: 108, dtype: float64

or

In [ ]:
exp_1.output['Gamma'].aggregate

## Plotting Sensitivity Measures

Depending on the sensitivity analysis problem at hand, several plots could be drawn. Two examples of them are included below:

In [ ]:
# Plotting IVARS-50 measure for two parameters

idx = pd.IndexSlice

exp_1.ivars.loc[idx[:, :, 0.5]].unstack(level=-1)[['C0', 'ETF', 'LP']].plot()

In [ ]:
# Plotting IVARS-50 measure using emperical cdfs for two parameters

import seaborn as sns

sns.ecdfplot(exp_1.ivars.loc[idx[:, :, 0.5]].unstack(level=-1)[['C0', 'ETF', 'LP']])

# DYI

How about you test other outputs of HBV-SASK yourself below?

In [ ]:
exp_2_model = Model(...)

exp_2 = TSVARS(...)